In [5]:
import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
import pandas as pd
import re

with open('H:\ML\movies.dat', 'r', encoding='latin1') as file:
    data = file.readlines()
    #"movieID", "title", "year", "genres"
movie_data = []
for line in data:
    parts = re.split(r'::|\(', line.strip())
    ID = int(parts[0])
    title = parts[1].strip()
    year_match = re.search(r'\d{4}', parts[2])
    year = int(year_match.group()) if year_match else None
    genres = parts[3].rstrip(')').split("|")
    movie_data.append([ID, title, year, genres])
    
columns = ["ID", "title", "year", "genres"]
movies_df = pd.DataFrame(movie_data, columns=columns)

def process_year(year):
    if pd.notna(year):  
        year_str = str(int(year))
        return year_str.replace(".", "").lstrip("0")
    return None

movies_df['year'] = movies_df['year'].apply(process_year)

In [7]:
with open(r'H:\ML\users.dat', 'r', encoding='latin1') as file:
    data = file.readlines()
  #UserID::Gender::Age::Occupation::Zip-code 
users_data = []
for line in data:
    parts = re.split(r'::|\(', line.strip())
    ID = int(parts[0])
    Gender = parts[1].strip()
    Age = int(parts[2])
    Occupation = parts[3].strip()
    Zip_code = parts[4].strip()
    users_data.append([ID, Gender, Age, Occupation, Zip_code])
    
columns = ["ID", "Gender", "Age", "Occupation", "Zip-code"]
users_df = pd.DataFrame(users_data, columns=columns)

In [8]:
with open(r'H:\ML\ratings.dat', 'r', encoding='latin1') as file:
    data = file.readlines()
#UserID::MovieID::Rating::Timestamp
rating_data = []

for line in data:
    parts = re.split(r'::|\(', line.strip())
    UserID = int(parts[0])
    MovieID = int(parts[1])
    Rating = int(parts[2])
    Timestamp = parts[3].strip()
    
    rating_data.append([UserID, MovieID, Rating, Timestamp])
    
columns = ["UserID", "MovieID", "Rating", "Timestamp"]
rating_df = pd.DataFrame(rating_data, columns=columns)


In [9]:
Allgenres= ['Action', 'Adventure', 'Animation', "Children's" , 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
'Sci-Fi', 'Thriller', 'War', 'Western']

In [10]:
#movies_df

In [11]:
for genre in Allgenres:
    movies_df[genre] = 0

In [12]:
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        if genre in Allgenres:
            movies_df.at[index, genre] = 1

In [13]:
#movies_df

In [14]:
#change_name = {"movieID": "MovieID"}
#movies_df.rename(columns=change_name, inplace=True)

In [15]:
#users_df

In [16]:
#rating_df

In [17]:
user_genres_df = rating_df.merge(movies_df, left_on= 'MovieID', right_on = 'ID', how ='inner').drop('ID',axis=1)

In [18]:
user_genres_df =user_genres_df.drop(['title','MovieID','year','Timestamp'],axis=1)

In [19]:
user_genres_df

,UserID,Rating,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,5,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,5,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,4,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,4,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,17,5,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,5,[Documentary],0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1000205,5675,3,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000206,5780,1,[Drama],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000207,5851,5,"[Comedy, Drama, Western]",0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [20]:
user_genres_df.drop(columns=['genres'], inplace=True)

In [21]:
user_genres_df.head()

,UserID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,12,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,15,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,17,5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [22]:
for genre in Allgenres:
    user_genres_df.loc[:, genre] = user_genres_df['Rating'] * user_genres_df[genre]

user_genres_df 

,UserID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
2,12,4,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0
3,15,4,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0
4,17,5,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,5,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0
1000205,5675,3,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0
1000206,5780,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1000207,5851,5,0,0,0,0,5,0,0,5,0,0,0,0,0,0,0,0,0,5


In [23]:
for col in user_genres_df.columns:
    user_genres_df[col+"_user_avg_Rating"] = user_genres_df[col]

In [24]:
user_genres_df

,UserID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,...,Fantasy_user_avg_Rating,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating
0,1,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
1,2,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
2,12,4,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
3,15,4,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
4,17,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,5,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
1000205,5675,3,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1000206,5780,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1000207,5851,5,0,0,0,0,5,0,0,5,...,0,0,0,0,0,0,0,0,0,5


In [25]:
user_genres_df["user_overall_avr_Rating"] = user_genres_df["Rating"]
user_genres_df = user_genres_df.drop("Rating",axis=1)
user_genres_df.head()

,UserID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,...,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
0,1,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5
1,2,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5
2,12,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,4
3,15,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,4
4,17,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5


In [26]:
user_genres_df.columns.tolist()

['UserID',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western',
 'UserID_user_avg_Rating',
 'Rating_user_avg_Rating',
 'Action_user_avg_Rating',
 'Adventure_user_avg_Rating',
 'Animation_user_avg_Rating',
 "Children's_user_avg_Rating",
 'Comedy_user_avg_Rating',
 'Crime_user_avg_Rating',
 'Documentary_user_avg_Rating',
 'Drama_user_avg_Rating',
 'Fantasy_user_avg_Rating',
 'Film-Noir_user_avg_Rating',
 'Horror_user_avg_Rating',
 'Musical_user_avg_Rating',
 'Mystery_user_avg_Rating',
 'Romance_user_avg_Rating',
 'Sci-Fi_user_avg_Rating',
 'Thriller_user_avg_Rating',
 'War_user_avg_Rating',
 'Western_user_avg_Rating',
 'user_overall_avr_Rating']

In [27]:
user_genres_df = user_genres_df.drop("UserID_user_avg_Rating",axis=1)

In [28]:
user_genres_df.head()

,UserID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,...,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
0,1,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5
1,2,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5
2,12,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,4
3,15,0,0,0,0,0,0,0,4,0,...,0,0,0,0,0,0,0,0,0,4
4,17,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,5


In [29]:
user_genres_df = user_genres_df.drop(Allgenres, axis=1)

In [30]:
user_genres_df.head()

,UserID,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,Crime_user_avg_Rating,Documentary_user_avg_Rating,Drama_user_avg_Rating,...,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
0,1,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,5
1,2,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,5
2,12,4,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,4
3,15,4,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,4
4,17,5,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,5


In [31]:
user_genres_df = user_genres_df.groupby('UserID').mean()

In [32]:
user_genres_df

,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,Crime_user_avg_Rating,Documentary_user_avg_Rating,Drama_user_avg_Rating,Fantasy_user_avg_Rating,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
UserID,,,,,,,,,,,,,,,,,,,,
1,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,0.075472,0.000000,1.754717,0.226415,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.000000,4.188679
2,3.713178,1.496124,0.550388,0.000000,0.000000,0.658915,0.333333,0.000000,2.356589,0.023256,0.031008,0.046512,0.000000,0.077519,0.658915,0.449612,0.837209,0.410853,0.100775,3.713178
3,3.901961,1.764706,1.941176,0.235294,0.235294,2.196078,0.000000,0.000000,0.627451,0.176471,0.000000,0.137255,0.078431,0.058824,0.372549,0.450980,0.372549,0.156863,0.549020,3.901961
4,4.190476,3.523810,1.095238,0.000000,0.190476,0.000000,0.000000,0.000000,1.190476,0.428571,0.000000,0.619048,0.000000,0.000000,0.142857,1.523810,0.666667,0.476190,0.428571,4.190476
5,3.146465,0.378788,0.136364,0.080808,0.116162,0.898990,0.292929,0.111111,1.525253,0.000000,0.060606,0.141414,0.050505,0.126263,0.419192,0.232323,0.535354,0.106061,0.020202,3.146465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3.302928,0.480856,0.270270,0.149775,0.209459,0.907658,0.191441,0.048423,1.353604,0.064189,0.072072,0.236486,0.129505,0.125000,0.431306,0.525901,0.511261,0.166667,0.057432,3.302928
6037,3.717822,0.480198,0.178218,0.019802,0.108911,0.995050,0.321782,0.019802,1.801980,0.084158,0.153465,0.183168,0.079208,0.237624,0.400990,0.702970,1.212871,0.301980,0.074257,3.717822
6038,3.800000,0.300000,0.200000,0.550000,0.150000,2.300000,0.000000,0.000000,1.750000,0.000000,0.000000,0.250000,0.000000,0.000000,1.250000,0.200000,0.000000,0.800000,0.000000,3.800000


In [33]:
user_genres_df.reset_index(inplace=True)

In [34]:
user_genres_df

,UserID,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,Crime_user_avg_Rating,Documentary_user_avg_Rating,Drama_user_avg_Rating,...,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
0,1,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,0.075472,0.000000,1.754717,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.000000,4.188679
1,2,3.713178,1.496124,0.550388,0.000000,0.000000,0.658915,0.333333,0.000000,2.356589,...,0.031008,0.046512,0.000000,0.077519,0.658915,0.449612,0.837209,0.410853,0.100775,3.713178
2,3,3.901961,1.764706,1.941176,0.235294,0.235294,2.196078,0.000000,0.000000,0.627451,...,0.000000,0.137255,0.078431,0.058824,0.372549,0.450980,0.372549,0.156863,0.549020,3.901961
3,4,4.190476,3.523810,1.095238,0.000000,0.190476,0.000000,0.000000,0.000000,1.190476,...,0.000000,0.619048,0.000000,0.000000,0.142857,1.523810,0.666667,0.476190,0.428571,4.190476
4,5,3.146465,0.378788,0.136364,0.080808,0.116162,0.898990,0.292929,0.111111,1.525253,...,0.060606,0.141414,0.050505,0.126263,0.419192,0.232323,0.535354,0.106061,0.020202,3.146465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,3.302928,0.480856,0.270270,0.149775,0.209459,0.907658,0.191441,0.048423,1.353604,...,0.072072,0.236486,0.129505,0.125000,0.431306,0.525901,0.511261,0.166667,0.057432,3.302928
6036,6037,3.717822,0.480198,0.178218,0.019802,0.108911,0.995050,0.321782,0.019802,1.801980,...,0.153465,0.183168,0.079208,0.237624,0.400990,0.702970,1.212871,0.301980,0.074257,3.717822
6037,6038,3.800000,0.300000,0.200000,0.550000,0.150000,2.300000,0.000000,0.000000,1.750000,...,0.000000,0.250000,0.000000,0.000000,1.250000,0.200000,0.000000,0.800000,0.000000,3.800000
6038,6039,3.878049,0.227642,0.333333,0.382114,0.487805,1.934959,0.065041,0.000000,0.772358,...,0.178862,0.032520,1.227642,0.577236,0.894309,0.276423,0.471545,0.300813,0.073171,3.878049


In [35]:
final_df = rating_df.merge(user_genres_df, on='UserID', how='inner')

In [36]:
final_df

,UserID,MovieID,Rating,Timestamp,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,...,Film-Noir_user_avg_Rating,Horror_user_avg_Rating,Musical_user_avg_Rating,Mystery_user_avg_Rating,Romance_user_avg_Rating,Sci-Fi_user_avg_Rating,Thriller_user_avg_Rating,War_user_avg_Rating,Western_user_avg_Rating,user_overall_avr_Rating
0,1,1193,5,978300760,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.00000,4.188679
1,1,661,3,978302109,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.00000,4.188679
2,1,914,3,978301968,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.00000,4.188679
3,1,3408,4,978300275,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.00000,4.188679
4,1,2355,5,978824291,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0.000000,0.000000,1.132075,0.000000,0.339623,0.245283,0.207547,0.188679,0.00000,4.188679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,3.577713,0.328446,0.181818,0.035191,0.070381,0.909091,...,0.117302,0.167155,0.105572,0.143695,0.410557,0.375367,0.460411,0.234604,0.02346,3.577713
1000205,6040,1094,5,956704887,3.577713,0.328446,0.181818,0.035191,0.070381,0.909091,...,0.117302,0.167155,0.105572,0.143695,0.410557,0.375367,0.460411,0.234604,0.02346,3.577713
1000206,6040,562,5,956704746,3.577713,0.328446,0.181818,0.035191,0.070381,0.909091,...,0.117302,0.167155,0.105572,0.143695,0.410557,0.375367,0.460411,0.234604,0.02346,3.577713
1000207,6040,1096,4,956715648,3.577713,0.328446,0.181818,0.035191,0.070381,0.909091,...,0.117302,0.167155,0.105572,0.143695,0.410557,0.375367,0.460411,0.234604,0.02346,3.577713


In [37]:
final_df = final_df.merge(movies_df, left_on='MovieID',right_on='ID', how='inner').drop('ID', axis=1)
final_df

,UserID,MovieID,Rating,Timestamp,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1193,5,978300760,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,...,0,0,0,0,0,0,0,0,0,0
1,2,1193,5,978298413,3.713178,1.496124,0.550388,0.000000,0.000000,0.658915,...,0,0,0,0,0,0,0,0,0,0
2,12,1193,4,978220179,3.826087,0.695652,0.608696,0.000000,0.217391,1.000000,...,0,0,0,0,0,0,0,0,0,0
3,15,1193,4,978199279,3.323383,0.925373,0.363184,0.039801,0.044776,0.940299,...,0,0,0,0,0,0,0,0,0,0
4,17,1193,5,978158471,4.075829,1.151659,0.677725,0.189573,0.222749,0.677725,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,2198,5,958846401,3.497238,0.359116,0.099448,0.066298,0.121547,1.248619,...,0,0,0,0,0,0,0,0,0,0
1000205,5675,2703,3,976029116,3.807167,0.926621,0.385666,0.087031,0.095563,1.054608,...,0,0,0,0,0,0,0,0,0,0
1000206,5780,2845,1,958153068,3.741313,1.463320,0.393822,0.073359,0.069498,0.760618,...,0,0,0,0,0,0,0,0,0,0
1000207,5851,3607,5,957756608,4.012346,1.098765,1.351852,0.796296,1.450617,0.759259,...,0,0,0,0,0,0,0,0,0,1


In [42]:
final_df = final_df.drop(columns=['MovieID','Timestamp','title', 'year','genres',])

In [43]:
final_df

,UserID,Rating,Rating_user_avg_Rating,Action_user_avg_Rating,Adventure_user_avg_Rating,Animation_user_avg_Rating,Children's_user_avg_Rating,Comedy_user_avg_Rating,Crime_user_avg_Rating,Documentary_user_avg_Rating,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,5,4.188679,0.320755,0.377358,1.396226,1.603774,1.094340,0.075472,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,2,5,3.713178,1.496124,0.550388,0.000000,0.000000,0.658915,0.333333,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,12,4,3.826087,0.695652,0.608696,0.000000,0.217391,1.000000,0.782609,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,15,4,3.323383,0.925373,0.363184,0.039801,0.044776,0.940299,0.492537,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,17,5,4.075829,1.151659,0.677725,0.189573,0.222749,0.677725,0.407583,0.023697,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,5949,5,3.497238,0.359116,0.099448,0.066298,0.121547,1.248619,0.795580,0.082873,...,0,0,0,0,0,0,0,0,0,0
1000205,5675,3,3.807167,0.926621,0.385666,0.087031,0.095563,1.054608,0.505119,0.006826,...,0,0,0,0,0,0,0,0,0,0
1000206,5780,1,3.741313,1.463320,0.393822,0.073359,0.069498,0.760618,0.795367,0.007722,...,0,0,0,0,0,0,0,0,0,0
1000207,5851,5,4.012346,1.098765,1.351852,0.796296,1.450617,0.759259,0.172840,0.030864,...,0,0,0,0,0,0,0,0,0,1


In [44]:
final_df.columns.tolist()

['UserID',
 'Rating',
 'Rating_user_avg_Rating',
 'Action_user_avg_Rating',
 'Adventure_user_avg_Rating',
 'Animation_user_avg_Rating',
 "Children's_user_avg_Rating",
 'Comedy_user_avg_Rating',
 'Crime_user_avg_Rating',
 'Documentary_user_avg_Rating',
 'Drama_user_avg_Rating',
 'Fantasy_user_avg_Rating',
 'Film-Noir_user_avg_Rating',
 'Horror_user_avg_Rating',
 'Musical_user_avg_Rating',
 'Mystery_user_avg_Rating',
 'Romance_user_avg_Rating',
 'Sci-Fi_user_avg_Rating',
 'Thriller_user_avg_Rating',
 'War_user_avg_Rating',
 'Western_user_avg_Rating',
 'user_overall_avr_Rating',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint
import pickle

X = final_df.drop('Rating', axis=1)
y = final_df['Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_dist = {
    'n_neighbors': randint(1, 50),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto']
}

knn = KNeighborsClassifier()

randomized_search = RandomizedSearchCV(estimator=knn, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=5, error_score='raise')
randomized_search.fit(X_train, y_train)

best_knn_model = randomized_search.best_estimator_
best_params = randomized_search.best_params_

accuracy = best_knn_model.score(X_test, y_test)
print("Best Model Accuracy:", accuracy)
print("Best Hyperparameters:", best_params)


pickle.dump(best_knn_model, open("model_clf_pickle", 'wb'))

my_model_clf = pickle.load(open("model_clf_pickle", 'rb'))
result_score = my_model_clf.score(X_test, y_test)
print("Score:", result_score)